In [1]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!mkdir -p ~/.kaggle
import json 
token = {"username":"seunghwan1228","key":"2a8fb35ecd319bf10a02713c306059ca"}
with open('/content/drive/My Drive/Colab Notebooks/Kaggle Kernel/kaggle.json', 'w') as file:
  json.dump(token, file)  
!cp drive/'My Drive'/'Colab Notebooks'/'Kaggle Kernel'/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls ~/.kaggle
!ls -l ~/.kaggle
!cat ~/.kaggle/kaggle.json


kaggle.json
total 4
-rw------- 1 root root 72 Nov 29 00:55 kaggle.json
{"username": "seunghwan1228", "key": "2a8fb35ecd319bf10a02713c306059ca"}

In [3]:
# down load path
# !kaggle competitions download -c jigsaw-unintended-bias-in-toxicity-classification
# !kaggle competitions download -c aptos2019-blindness-detection
# !kaggle datasets download -d moltean/fruits
!kaggle competitions download -c quora-question-pairs

100% 4.95M/4.95M [00:00<00:00, 26.7MB/s]

 43% 9.00M/21.2M [00:01<00:02, 5.00MB/s]
100% 21.2M/21.2M [00:01<00:00, 13.7MB/s]
 86% 97.0M/112M [00:04<00:01, 14.8MB/s]
100% 112M/112M [00:04<00:00, 28.9MB/s] 


In [4]:
!unzip train.csv.zip
!unzip test.csv.zip

Archive:  train.csv.zip
  inflating: train.csv               
Archive:  test.csv.zip
  inflating: test.csv                


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.preprocessing import text, sequence
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [0]:
train = pd.read_csv('/content/train.csv')

In [3]:
train.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [0]:
courpus = pd.concat([train['question1'], train['question2']]).reset_index()

In [5]:
courpus.head()

,index,0
0,0,What is the step by step guide to invest in sh...
1,1,What is the story of Kohinoor (Koh-i-Noor) Dia...
2,2,How can I increase the speed of my internet co...
3,3,Why am I mentally very lonely? How can I solve...
4,4,"Which one dissolve in water quikly sugar, salt..."


In [6]:
courpus[0][0]

'What is the step by step guide to invest in share market in india?'

In [0]:
import re
re_pattern = re.compile('[^a-zA-Z0-9]')

def remove_puct(x):
  x = str(x)
  x = re.sub(re_pattern, ' ', x)
  return x


In [8]:
remove_puct(courpus[0][1])

'What is the story of Kohinoor  Koh i Noor  Diamond '

In [0]:
courpus = courpus.astype('str')

In [0]:
courpus[0] = courpus[0].apply(remove_puct)

In [0]:
courpus = courpus[0].to_list()

## Use Soy NLP

In [12]:
!pip install soynlp

In [0]:
with open('courpus.txt', 'w') as f:
  for i in courpus:
    f.write(str(i) + '\n')

In [0]:
from soynlp.word import WordExtractor

In [0]:
word_ext = WordExtractor(min_frequency=10, min_cohesion_forward=0.05)

In [16]:
word_ext.train(courpus)

training was done. used memory 0.868 Gb


In [17]:
word_ext.word_scores()

all cohesion probabilities was computed. # words = 85975
all branching entropies was computed # words = 136933
all accessor variety was computed # words = 136933


{'concus': Scores(cohesion_forward=0.14408995338225414, cohesion_backward=0, left_branching_entropy=0, right_branching_entropy=-0.0, left_accessor_variety=0, right_accessor_variety=1, leftside_frequency=24, rightside_frequency=0),
 'Jinpin': Scores(cohesion_forward=0.27423495848148755, cohesion_backward=0, left_branching_entropy=0, right_branching_entropy=-0.0, left_accessor_variety=0, right_accessor_variety=1, leftside_frequency=31, rightside_frequency=0),
 'coni': Scores(cohesion_forward=0.03726739925139639, cohesion_backward=0.07156953464970622, left_branching_entropy=1.1490596969706202, right_branching_entropy=1.0789922078775833, left_accessor_variety=4, right_accessor_variety=3, leftside_frequency=20, rightside_frequency=11),
 'mpian': Scores(cohesion_forward=0, cohesion_backward=0.0606215261351843, left_branching_entropy=-0.0, right_branching_entropy=1.0608569471580214, left_accessor_variety=1, right_accessor_variety=3, leftside_frequency=0, rightside_frequency=11),
 'adapt': Sco

In [0]:
import math
from soynlp.tokenizer import LTokenizer

In [19]:
score = word_ext.word_scores()

all cohesion probabilities was computed. # words = 85975
all branching entropies was computed # words = 136933
all accessor variety was computed # words = 136933


In [0]:
scores = {key:(score[key].cohesion_forward * math.exp(score[key].right_branching_entropy)) for key in score.keys()}

In [0]:
tokenizer = LTokenizer(scores)

In [22]:
tokenizer.tokenize(courpus[0])

['What',
 'is',
 'the',
 'st',
 'ep',
 'by',
 'st',
 'ep',
 'guide',
 'to',
 'in',
 'vest',
 'in',
 'share',
 'market',
 'in',
 'in',
 'dia']

In [0]:
from tqdm import tqdm_notebook

In [24]:
len(courpus)

808580

In [0]:
courpus_df = pd.DataFrame(courpus)

In [0]:
courpus_df['soynlp'] = courpus_df[0].apply(lambda x: tokenizer.tokenize(x))

In [27]:
courpus_df.head()

,0,soynlp
0,What is the step by step guide to invest in sh...,"[What, is, the, st, ep, by, st, ep, guide, to,..."
1,What is the story of Kohinoor Koh i Noor Dia...,"[What, is, the, st, ory, of, Kohinoor, Ko, h, ..."
2,How can I increase the speed of my internet co...,"[How, can, I, in, crease, the, speed, of, my, ..."
3,Why am I mentally very lonely How can I solve...,"[Wh, y, am, I, me, ntally, ver, y, lonely, How..."
4,Which one dissolve in water quikly sugar salt...,"[Which, on, e, dissolve, in, water, qu, ikly, ..."


In [0]:
courpus_cleaned = courpus_df['soynlp'].to_list()

In [29]:
courpus_cleaned

[['What',
  'is',
  'the',
  'st',
  'ep',
  'by',
  'st',
  'ep',
  'guide',
  'to',
  'in',
  'vest',
  'in',
  'share',
  'market',
  'in',
  'in',
  'dia'],
 ['What',
  'is',
  'the',
  'st',
  'ory',
  'of',
  'Kohinoor',
  'Ko',
  'h',
  'i',
  'No',
  'or',
  'Diamond'],
 ['How',
  'can',
  'I',
  'in',
  'crease',
  'the',
  'speed',
  'of',
  'my',
  'in',
  'ternet',
  'connect',
  'ion',
  'while',
  'using',
  'a',
  'VPN'],
 ['Wh',
  'y',
  'am',
  'I',
  'me',
  'ntally',
  'ver',
  'y',
  'lonely',
  'How',
  'can',
  'I',
  'solve',
  'it'],
 ['Which',
  'on',
  'e',
  'dissolve',
  'in',
  'water',
  'qu',
  'ikly',
  'sugar',
  'salt',
  'me',
  'thane',
  'and',
  'carbon',
  'di',
  'oxide'],
 ['Astrology',
  'I',
  'am',
  'a',
  'Capricorn',
  'Sun',
  'Ca',
  'p',
  'moon',
  'and',
  'ca',
  'p',
  'rising',
  'what',
  'does',
  'that',
  'say',
  'about',
  'me'],
 ['Should', 'I', 'buy', 'ti', 'ago'],
 ['How', 'can', 'I', 'be', 'a', 'good', 'ge', 'ologist'],
 

In [0]:
k_token = text.Tokenizer(oov_token='UNK')

In [0]:
k_token.fit_on_texts(courpus_cleaned)

In [0]:
max_len = len(k_token.index_word)

In [33]:
train

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
...,...,...,...,...,...,...
404285,404285,433578,379845,How many keywords are there in the Racket prog...,How many keywords are there in PERL Programmin...,0
404286,404286,18840,155606,Do you believe there is life after death?,Is it true that there is life after death?,1
404287,404287,537928,537929,What is one coin?,What's this coin?,0
404288,404288,537930,537931,What is the approx annual cost of living while...,I am having little hairfall problem but I want...,0


In [34]:
train[['question1', 'question2']]

,question1,question2
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...
2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...
3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...
4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?
...,...,...
404285,How many keywords are there in the Racket prog...,How many keywords are there in PERL Programmin...
404286,Do you believe there is life after death?,Is it true that there is life after death?
404287,What is one coin?,What's this coin?
404288,What is the approx annual cost of living while...,I am having little hairfall problem but I want...


In [0]:
x = train[['question1', 'question2']]
y = train['is_duplicate']

In [0]:
x_q1 = x['question1'].astype('str')
x_q2 = x['question2'].astype('str')

In [0]:
x_q1_tk = k_token.texts_to_sequences(x_q1)
x_q2_tk = k_token.texts_to_sequences(x_q2)

In [0]:
x_q1_pad = sequence.pad_sequences(x_q1_tk, maxlen=200, padding='post')
x_q2_pad = sequence.pad_sequences(x_q2_tk, maxlen=200, padding='post')

In [39]:
x_q1_pad[0].shape

(200,)

In [0]:
x = np.concatenate([x_q1_pad, x_q2_pad], axis=1)

In [0]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=1228)

In [0]:
x_train_q1 = x_train[:, :200]
x_train_q2 = x_train[:, 200:]

x_test_q1 = x_test[:, :200]
x_test_q2 = x_test[:, 200:]

In [0]:
assert x_train_q1.all() == x_q1_pad.all()

In [0]:
from keras import models, layers

## ManDistance



In [0]:
from keras import backend as K
def man_dist_lambda(left, right):
  return K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True))

In [46]:
emb = layers.Embedding(max_len+1, 300, input_length=200)
l_lstm = layers.Bidirectional(layers.CuDNNLSTM(512))

q1_input = layers.Input(shape=(200,))
q2_input = layers.Input(shape=(200,))

q1 = emb(q1_input)
q2 = emb(q2_input)

l1 = l_lstm(q1)
l2 = l_lstm(q2)

man_dist = layers.Lambda(lambda x: man_dist_lambda(x[0], x[1]))([l1,l2])

malstm = models.Model([q1_input, q2_input], man_dist)

In [0]:
from keras import optimizers
optimizer = optimizers.Adadelta(clipnorm=1.25)

In [48]:
malstm.compile(loss='mse', optimizer=optimizer, metrics=['acc', 'mae'])

In [0]:
from tensorboardcolab import * 

In [50]:
tbc = TensorBoardColab()

Wait for 8 seconds...
TensorBoard link:
http://3d92c557.ngrok.io


In [0]:
from keras import backend as K

In [0]:
model_1 = malstm.fit([x_train_q1, x_train_q2], y_train, 
                     epochs=100, 
                     batch_size=512, 
                     validation_data=([x_test_q1, x_test_q2], y_test), 
                     callbacks=[TensorBoardColabCallback(tbc)])




Train on 323432 samples, validate on 80858 samples







Epoch 1/100
323432/323432 [==============================] - 486s 2ms/step - loss: 0.2090 - acc: 0.6651 - mean_absolute_error: 0.3982 - val_loss: 0.1930 - val_acc: 0.6924 - val_mean_absolute_error: 0.3847

Epoch 2/100
323432/323432 [==============================] - 483s 1ms/step - loss: 0.1866 - acc: 0.7113 - mean_absolute_error: 0.3771 - val_loss: 0.1812 - val_acc: 0.7235 - val_mean_absolute_error: 0.3713
Epoch 3/100
323432/323432 [==============================] - 483s 1ms/step - loss: 0.1772 - acc: 0.7349 - mean_absolute_error: 0.3646 - val_loss: 0.1785 - val_acc: 0.7359 - val_mean_absolute_error: 0.3663
Epoch 4/100
323432/323432 [==============================] - 484s 1ms/step - loss: 0.1717 - acc: 0.7482 - mean_absolute_error: 0.3566 - val_loss: 0.1733 - val_acc: 0.7417 - val_mean_absolute_error: 0.3534
Epoch 5/100
323432/323432 [==============================] - 484s 1ms/step - loss: 0.1679 - acc: 0.7575 - mean_absolut

# prediction

In [0]:
train_shape = 323432

def convert_query(x, train_shape):
  '''
  Convert query for model, and repeat whole number of comparison
  '''
  %%time
  x = tokenizer.tokenize(str(x))
  x = k_token.texts_to_sequences([x])
  x = sequence.pad_sequences(x, maxlen=200, padding='post')
  x = np.tile(x, (train_shape, 1))
  return x

In [0]:
# test query

result = convert_query('how can be the data scientist', train_shape=train_shape)

In [0]:
result

In [0]:
result.shape

In [0]:
import gc
gc.collect()